In [1]:
import torch
import numpy as np

In [2]:
def i2b(A):
    return torch.tensor(list(map(lambda x : list([int(z) for z in '{0:>04b}'.format(x)]), A)))

def encode(U, G):
    return torch.matmul(Uhg, G)%2

def mod_bpsk(C):
    X = -(2*C-1)
    return X

def awgn(X, SNR):
    Z = torch.randn((X.shape[0], 7))/np.sqrt(SNR)
    Y = X + Z
    return Y

def decode_ml(y, CB):
    indx = torch.argmin((torch.abs(Y.unsqueeze(1) - CB)**2).sum(axis=2), dim=1)
    U_hat = CB[indx]
    return U_hat

In [4]:
G = torch.tensor([[1, 0, 0, 0, 1, 1, 0],
                  [0, 1, 0, 0, 1, 0, 1],
                  [0, 0, 1, 0, 0, 1, 1],
                  [0, 0, 0, 1, 1, 1, 1]])

H = torch.tensor([[1, 1, 0, 1, 1, 0, 0],
                  [1, 0, 1, 1, 0, 1, 0],
                  [0, 1, 1, 1, 0, 0, 1]])

i = torch.arange(0,16)
I = i2b(i)

CB = mod_bpsk(torch.matmul(I, G)%2)

In [5]:
I, CB

(tensor([[0, 0, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
         [0, 0, 1, 1],
         [0, 1, 0, 0],
         [0, 1, 0, 1],
         [0, 1, 1, 0],
         [0, 1, 1, 1],
         [1, 0, 0, 0],
         [1, 0, 0, 1],
         [1, 0, 1, 0],
         [1, 0, 1, 1],
         [1, 1, 0, 0],
         [1, 1, 0, 1],
         [1, 1, 1, 0],
         [1, 1, 1, 1]]),
 tensor([[ 1,  1,  1,  1,  1,  1,  1],
         [ 1,  1,  1, -1, -1, -1, -1],
         [ 1,  1, -1,  1,  1, -1, -1],
         [ 1,  1, -1, -1, -1,  1,  1],
         [ 1, -1,  1,  1, -1,  1, -1],
         [ 1, -1,  1, -1,  1, -1,  1],
         [ 1, -1, -1,  1, -1, -1,  1],
         [ 1, -1, -1, -1,  1,  1, -1],
         [-1,  1,  1,  1, -1, -1,  1],
         [-1,  1,  1, -1,  1,  1, -1],
         [-1,  1, -1,  1, -1,  1, -1],
         [-1,  1, -1, -1,  1, -1,  1],
         [-1, -1,  1,  1,  1, -1, -1],
         [-1, -1,  1, -1, -1,  1,  1],
         [-1, -1, -1,  1,  1,  1,  1],
         [-1, -1, -1, -1, -1, -1, -1]]))

In [6]:
u = torch.randint(0, 16, (10000,))
U = i2b(u)
u, U

(tensor([ 1, 11,  1,  ..., 15, 13,  6]),
 tensor([[0, 0, 0, 1],
         [1, 0, 1, 1],
         [0, 0, 0, 1],
         ...,
         [1, 1, 1, 1],
         [1, 1, 0, 1],
         [0, 1, 1, 0]]))

In [7]:
(C := encode(U, G), C.shape)

(tensor([[0, 0, 0,  ..., 1, 1, 1],
         [1, 0, 1,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 0,  ..., 1, 0, 0],
         [0, 1, 1,  ..., 1, 1, 0]]),
 torch.Size([10000, 7]))

In [8]:
(X := mod_bpsk(C), X.shape)

(tensor([[ 1,  1,  1,  ..., -1, -1, -1],
         [-1,  1, -1,  ...,  1, -1,  1],
         [ 1,  1,  1,  ..., -1, -1, -1],
         ...,
         [-1, -1, -1,  ..., -1, -1, -1],
         [-1, -1,  1,  ..., -1,  1,  1],
         [ 1, -1, -1,  ..., -1, -1,  1]]),
 torch.Size([10000, 7]))

In [9]:
(Y := awgn(X, 0.5), Y.shape)

(tensor([[ 0.5567,  0.1186,  3.7129,  ..., -1.9041,  0.5170, -3.2673],
         [-1.3063,  0.2938, -0.1889,  ...,  1.6802, -0.8313,  3.0304],
         [ 1.1673,  3.0264,  0.0542,  ..., -1.4620, -3.5965, -1.0669],
         ...,
         [-0.2274, -2.0577, -0.7597,  ..., -0.1960, -1.2832, -0.9709],
         [-2.2854, -3.8564,  1.4052,  ..., -2.1198,  2.4428,  3.2940],
         [-0.0660, -0.0061, -1.2148,  ...,  0.4069,  0.2509,  1.0553]]),
 torch.Size([10000, 7]))

In [10]:
(U_hat := decode_ml(Y,CB), U_hat.shape)

(tensor([[ 1,  1,  1,  ..., -1, -1, -1],
         [-1,  1, -1,  ...,  1, -1,  1],
         [ 1,  1,  1,  ..., -1, -1, -1],
         ...,
         [-1, -1, -1,  ..., -1, -1, -1],
         [-1, -1,  1,  ..., -1,  1,  1],
         [-1,  1, -1,  ...,  1, -1,  1]]),
 torch.Size([10000, 7]))

In [11]:
(X != U_hat).sum()/(10000*7)

tensor(0.2206)